In [2]:
from pyo import *
import time

# Usa la configuración que te funcionó
s = Server().boot()
s.start()

# Fuente de audio
sf = SfPlayer("audios/drugs.mp3", speed=1.0, loop=True)

# Análisis FFT y cambio de tono
fft = PVAnal(sf, size=1024)
pitch = PVTranspose(fft, transpo=1.0)

# Síntesis con control de volumen
synth = PVSynth(pitch)
vol = Sig(0.5)  # volumen inicial
amp = synth * vol
amp.out()

# Cambios durante la ejecución
print("🎧 Reproduciendo: speed=1.0, pitch=0, vol=0.5")
time.sleep(3)

print("⚡ Subiendo velocidad a 1.5 (tempo + tono)")
sf.speed = 1.5
time.sleep(3)

print("🎵 Cambiando pitch a -4 semitonos")
pitch.transpo = -4
time.sleep(3)

print("🔉 Bajando volumen a 0.2")
vol.value = 0.2
time.sleep(3)

print("🔁 Restaurando valores originales")
sf.speed = 1.0
pitch.transpo = 0
vol.value = 1.0
time.sleep(4)

s.stop()
s.shutdown()


KeyboardInterrupt: 

In [17]:
from pyo import *
import time

# Initialize the audio server
s = Server(audio="portaudio")  # Use PortAudio for cross-platform compatibility
s.boot()
s.start()

# Method 1: Using audio input (microphone) to capture what's playing
# This captures whatever audio is coming through your microphone
input_signal = Input(chnl=0, mul=0.7)  # Get audio from input channel 0

# Method 2: If you want to load and play an audio file directly
# sf = SfPlayer("path/to/your/audiofile.wav", loop=True, mul=0.5)
# input_signal = sf

# Create spectrum analyzer
spectrum = Spectrum(
    input=input_signal,
    size=1024,           # FFT size - larger = better frequency resolution
    wintype=2            # Hanning window (good general purpose choice)
)

# Function to capture spectrum data
spectrum_data = []

def capture_spectrum_data(data):
    """
    Callback function to capture spectrum data
    data is a list of lists (one per channel)
    """
    global spectrum_data
    spectrum_data = data
    print(f"Captured spectrum frame with {len(data)} channels")
    if data:
        print(f"First channel has {len(data[0])} frequency bins")
        # You can process the data here or store it for later use

# Set the callback function to capture data
spectrum.setFunction(capture_spectrum_data)

# Configure spectrum analysis parameters
spectrum.setLowFreq(20)      # Low frequency bound (Hz)
spectrum.setHighFreq(20000)  # High frequency bound (Hz)
spectrum.polltime(0.05)      # Update rate (20 times per second)
spectrum.poll(True)          # Start polling

# Optional: Open visual display window
spectrum.view(title="Live Spectrum Analysis")

# Keep the analysis running
print("Starting live spectrum analysis...")
print("Press Ctrl+C to stop")

try:
    while True:
        time.sleep(0.1)
        # Your spectrum data is continuously updated in spectrum_data
        # You can process it here or in the callback function
        
except KeyboardInterrupt:
    print("\nStopping analysis...")
    spectrum.poll(False)
    s.stop()
    s.shutdown()